# ARIMA Data Preprocessing

In [1]:
import pandas as pd
import pyarrow as pa

# Read chartevents_subset from parquet file to pandas data frame
chartevents_resampled = pd.read_parquet('./data/chartevents_resampled.parquet', engine='pyarrow')

In [2]:
PARAMETER = 'HR'
#CHUNKS = chartevents.CHUNK_ID_FILLED_TH.unique()
CHUNKS = ['296490.0_220045.0_2192-09-26 23:51:00']

# Sampling rate of 1 data point per hour - Test for different values in the future - e.g. longer training set
TRAIN = 12 # 12 * 1 h = 12 hour training period
TEST = 1 # 1 * 1 h = 2 hours testing period
STEP = 1 # move 1 * 1 h = 1 hour per step

In [3]:
# Subset data based on PARAMETER & CHUNKS
arima_data = chartevents_resampled[
    (chartevents_resampled['VITAL_PARAMETER_NAME'] == PARAMETER) & 
    (chartevents_resampled.CHUNK_ID_FILLED_TH.isin(CHUNKS))
    ][['CHUNK_ID_FILLED_TH','CHARTTIME','VITAL_PARAMETER_NAME','VITAL_PARAMTER_VALUE_MEDIAN_RESAMPLING','VITAL_PARAMTER_VALUE_MEAN_RESAMPLING','VITAL_PARAMTER_VALUE_MAX_RESAMPLING','VITAL_PARAMTER_VALUE_MIN_RESAMPLING','THRESHOLD_VALUE_HIGH','THRESHOLD_VALUE_LOW']]

In [4]:
# Filter for chunks that have sufficient values to be used for training and testing the model
all_chunks_value_count = arima_data.CHUNK_ID_FILLED_TH.value_counts()
chunkid_filter = all_chunks_value_count[all_chunks_value_count >= (TRAIN + TEST)].index
arima_data = arima_data[arima_data.CHUNK_ID_FILLED_TH.isin(chunkid_filter)]


In [5]:
# Create new HOURS_SINCE_FIRST_RECORD column containing the time difference that has passed since the first timestamp of the measurement series.
import numpy as np
# arima_data['MINUTES_SINCE_FIRST_RECORD'] = arima_data.groupby('CHUNK_ID_FILLED_TH')#['CHARTTIME'].transform(lambda x: (x - x.min())/np.timedelta64(1,'m'))
# Alternative for hours instead of minutes
arima_data['HOURS_SINCE_FIRST_RECORD'] = arima_data.groupby('CHUNK_ID_FILLED_TH')['CHARTTIME'].transform(lambda x: (x - x.min())/np.timedelta64(1,'h'))


In [6]:
arima_data = arima_data[:15]

### First Adaption
Create dict that holds vital parameter series, threshold high and threshold low series for each chunk id (key). The series are all indexed the same way (= dif to first measurement in hours with current sampling rate) so they relate to the same time

In [7]:
# Create dictionary with chunk id as key and a dataframe as value.
# This dataframe contains of three columns the vital parameter values, the high thresholds and the low thresholds.
# As the index of these three list is the same and can be referenced back to the "HOURS_SINCE_FIRST_RECORD", we keep the time related information.
# Example:
# dict_of_chunk_series = {
#     "<chunkid_A>" : { | vital_parameter_series | threshold_high_series | threshold_low_series
#                     0 |                   95.0 |                   120 |                   60
#                     1 |                   90.5 |                   120 |                   60
#                     2 |                   91.0 |                   120 |                   60
#                    },
#     "<chunkid_B>" : { | vital_parameter_series | threshold_high_series | threshold_low_series
#                     0 |                   88.0 |                   110 |                   50
#                     1 |                   78.5 |                   110 |                   50
#                     2 |                   73.0 |                   120 |                   60
#                    }
#  }
#
# Example with additional vital parameter series which can be used as an exogenous variable for ARIMAX
# dict_of_chunk_series_with_test_and_train = {
#     "<chunkid_A>" : | vital_parameter_series_median | vital_parameter_series_mean | vital_parameter_series_min |threshold_high_series | threshold_low_series (+max)
#                   0 |                          95.0 |                        98.0 |                          80|                  120 |                   60
#                   1 |                          90.5 |                        96.0 |                          79|                  120 |                   60
#                   2 |                          91.0 |                        94.0 |                          83|                  120 |                   60
#  }

dict_of_chunk_series = {}

for chunkid in chunkid_filter:
    
    chunk_data = arima_data[arima_data.CHUNK_ID_FILLED_TH == chunkid].copy()

    # vital parameter series - median resampling
    chunk_value_series_median = pd.Series(chunk_data['VITAL_PARAMTER_VALUE_MEDIAN_RESAMPLING'],name="vital_parameter_series_median")
    chunk_value_series_median = chunk_value_series_median.reset_index(drop=True)
    chunk_value_series_median.index = list(chunk_value_series_median.index)

    # vital parameter series - mean resampling
    chunk_value_series_mean = pd.Series(chunk_data['VITAL_PARAMTER_VALUE_MEAN_RESAMPLING'],name="vital_parameter_series_mean")
    chunk_value_series_mean = chunk_value_series_mean.reset_index(drop=True)
    chunk_value_series_mean.index = list(chunk_value_series_mean.index)

    # vital parameter series - min resampling
    chunk_value_series_min = pd.Series(chunk_data['VITAL_PARAMTER_VALUE_MIN_RESAMPLING'],name="vital_parameter_series_min")
    chunk_value_series_min = chunk_value_series_min.reset_index(drop=True)
    chunk_value_series_min.index = list(chunk_value_series_min.index)

    # vital parameter series - max resampling
    chunk_value_series_max = pd.Series(chunk_data['VITAL_PARAMTER_VALUE_MAX_RESAMPLING'],name="vital_parameter_series_max")
    chunk_value_series_max = chunk_value_series_max.reset_index(drop=True)
    chunk_value_series_max.index = list(chunk_value_series_max.index)  
    

    # threshold series high
    chunk_threshold_high_series = pd.Series(chunk_data['THRESHOLD_VALUE_HIGH'],name="threshold_high_series")
    chunk_threshold_high_series = chunk_threshold_high_series.reset_index(drop=True)
    chunk_threshold_high_series.index = list(chunk_threshold_high_series.index)

    # threshold series low
    chunk_threshold_low_series = pd.Series(chunk_data['THRESHOLD_VALUE_LOW'],name="threshold_low_series")
    chunk_threshold_low_series = chunk_threshold_low_series.reset_index(drop=True)
    chunk_threshold_low_series.index = list(chunk_threshold_low_series.index)

    # Append series with key (CHUNK_ID) into dictionary
    vital_parameter_and_thresholds_for_chunkid = pd.concat([chunk_value_series_median,chunk_value_series_mean,chunk_value_series_min,chunk_value_series_max,chunk_threshold_high_series,chunk_threshold_low_series],axis=1)
    dict_of_chunk_series[chunkid] = vital_parameter_and_thresholds_for_chunkid

In [8]:
import pickle
# Rausschreiben
output_file = open('dict_of_chunk_series.pickle', 'wb')
pickle.dump(dict_of_chunk_series, output_file)
output_file.close()

In [9]:
# Einlesen
input_file = open('dict_of_chunk_series.pickle', 'rb')
dict_of_chunk_series = pickle.load(input_file)
input_file.close()
dict_of_chunk_series

{'296490.0_220045.0_2192-09-26 23:51:00':     vital_parameter_series_median  vital_parameter_series_mean  \
 0                            95.0                    95.000000   
 1                            90.5                    90.766667   
 2                            91.0                    90.083333   
 3                            91.0                    90.333333   
 4                            85.0                    86.016667   
 5                            89.0                    87.883333   
 6                            82.0                    81.766667   
 7                            80.0                    80.916667   
 8                            77.0                    77.233333   
 9                            78.5                    79.033333   
 10                           78.0                    78.250000   
 11                           84.0                    84.766667   
 12                           75.0                    76.400000   
 13                  

### Second Adaption
Specific TRAIn and TEST Size

In [10]:
# Create multiple test & train sets for each chunk to iteratively predict the next x measurements
# Create nested dictionary that holds the CHUNK_ID as first key.
# This key holds one dictionary for each iteration over this chunk. This depends on the TEST, TRAIN, and STEP.
# For each iteration we create another dictionary, whereby the last index of the train list acts as key.
# This key holds again one dictionary for the train list and one for the test list.
# Example:
# dict_of_chunk_series_with_test_and_train = {
#     "<chunkid_A>" : {
#         "<last_index_of_training_list_of_first_chunkid_A_iteration>" : {
#             "TRAIN_LIST_MEDIAN" : train_list_median,
#             "TEST_LIST_MEDIAN" : test_list_median,
#             "TRAIN_LIST_MEAN" : train_list_mean,
#             "TEST_LIST_MEAN" : test_list_mean,
#             "TRAIN_LIST_MIN" : train_list_min,
#             "TEST_LIST_MIN" : test_list_min,
#             "TRAIN_LIST_MAX" : train_list_max,
#             "TEST_LIST_MAX" : test_list_max,
#             "THRESHOLD_HIGH_FOR_TRAIN_LIST" : threshold_high_for_train_list ,
#             "THRESHOLD_LOW_FOR_TEST_LIST" : threshold_low_for_test_list
#         },
#         "<last_index_of_training_list_of_second_chunkid_A_iteration>" : {
#             "TRAIN_LIST_MEDIAN" : train_list_median,
#             "TEST_LIST_MEDIAN" : test_list_median,
#             "TRAIN_LIST_MEAN" : train_list_mean,
#             "TEST_LIST_MEAN" : test_list_mean,
#             "TRAIN_LIST_MIN" : train_list_min,
#             "TEST_LIST_MIN" : test_list_min,
#             "TRAIN_LIST_MAX" : train_list_max,
#             "TEST_LIST_MAX" : test_list_max,
#             "THRESHOLD_HIGH_FOR_TRAIN_LIST" : threshold_high_for_train_list ,
#             "THRESHOLD_LOW_FOR_TEST_LIST" : threshold_low_for_test_list
#         },
#     }
# }

dict_of_chunk_series_with_test_and_train = {}

for i, chunk in enumerate(dict_of_chunk_series):
    # acces dataframe of current chunk
    chunk_series_for_chunk = dict_of_chunk_series[chunk]

    # access vital_parameter_series_median of current chunk
    chunk_value_series_for_chunk_median = chunk_series_for_chunk["vital_parameter_series_median"]
    # access vital_parameter_series_mean of current chunk
    chunk_value_series_for_chunk_mean = chunk_series_for_chunk["vital_parameter_series_mean"]
    # access vital_parameter_series_min of current chunk
    chunk_value_series_for_chunk_min = chunk_series_for_chunk["vital_parameter_series_min"]
    # access vital_parameter_series_max of current chunk
    chunk_value_series_for_chunk_max = chunk_series_for_chunk["vital_parameter_series_max"]
        
    # access threshold_high_series of current chunk
    chunk_threshold_high_series_for_chunk = chunk_series_for_chunk["threshold_high_series"]
    # access threshold_low_series of current chunk
    chunk_threshold_low_series_for_chunk = chunk_series_for_chunk["threshold_low_series"]

    # create an empty dictionary for the key of the current chunk
    dict_of_chunk_series_with_test_and_train[chunk] = {}

    # create multiple test and train lists for that chunk
    for start in range(0, len(chunk_value_series_for_chunk_median) - (TRAIN + TEST)+1, STEP):
        
        # vital_parameter_series_median
        train_list_median = pd.Series(chunk_value_series_for_chunk_median[start : start+TRAIN], name="train_list_median")
        test_list_median = pd.Series(chunk_value_series_for_chunk_median[start+TRAIN : start+TRAIN+TEST], name="test_list_median")
        # vital_parameter_series_mean
        train_list_mean = pd.Series(chunk_value_series_for_chunk_mean[start : start+TRAIN], name="train_list_mean")
        test_list_mean = pd.Series(chunk_value_series_for_chunk_mean[start+TRAIN : start+TRAIN+TEST], name="test_list_mean")
        # vital_parameter_series_min
        train_list_min = pd.Series(chunk_value_series_for_chunk_min[start : start+TRAIN], name="train_list_min")
        test_list_min = pd.Series(chunk_value_series_for_chunk_min[start+TRAIN : start+TRAIN+TEST], name="test_list_min")
        # vital_parameter_series_max
        train_list_max = pd.Series(chunk_value_series_for_chunk_max[start : start+TRAIN], name="train_list_max")
        test_list_max = pd.Series(chunk_value_series_for_chunk_max[start+TRAIN : start+TRAIN+TEST], name="test_list_max")
        
        #threshold series high & low
        threshold_high_for_test_list = pd.Series(chunk_threshold_high_series_for_chunk[start+TRAIN : start+TRAIN+TEST],name="threshold_high_for_test_list")
        threshold_low_for_test_list = pd.Series(chunk_threshold_low_series_for_chunk[start+TRAIN : start+TRAIN+TEST],name="threshold_low_for_test_list")
        
        # For each iteration over the current chunk, we will create a dictionary that holds again the test and train list as dictionary
        # We use the last index of the current train list (which currently refers to the difference to first measurement) as second key
        second_key = train_list_median.index.max()
        dict_of_chunk_series_with_test_and_train[chunk][second_key] = {}
        # Assign the train and test list to the current chunk iteration      
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TRAIN_LIST_MEDIAN"] = train_list_median
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TRAIN_LIST_MEAN"] = train_list_mean
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TRAIN_LIST_MIN"] = train_list_min
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TRAIN_LIST_MAX"] = train_list_max

        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TEST_LIST_MEDIAN"] = test_list_median
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TEST_LIST_MEAN"] = test_list_median
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TEST_LIST_MIN"] = test_list_min
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["TEST_LIST_MAX"] = test_list_max

        dict_of_chunk_series_with_test_and_train[chunk][second_key]["THRESHOLD_HIGH_FOR_TEST_LIST"] = threshold_high_for_test_list
        dict_of_chunk_series_with_test_and_train[chunk][second_key]["THRESHOLD_LOW_FOR_TEST_LIST"] = threshold_low_for_test_list

In [11]:
import pickle
# Rausschreiben
second_output_file = open('dict_of_chunk_series_with_test_and_train.pickle', 'wb')
pickle.dump(dict_of_chunk_series_with_test_and_train, second_output_file)
second_output_file.close()

In [12]:
# Einlesen
second_input_file = open('dict_of_chunk_series_with_test_and_train.pickle', 'rb')
dict_of_chunk_series_with_test_and_train = pickle.load(second_input_file)
second_input_file.close()
dict_of_chunk_series_with_test_and_train

{'296490.0_220045.0_2192-09-26 23:51:00': {11: {'TRAIN_LIST_MEDIAN': 0     95.0
   1     90.5
   2     91.0
   3     91.0
   4     85.0
   5     89.0
   6     82.0
   7     80.0
   8     77.0
   9     78.5
   10    78.0
   11    84.0
   Name: train_list_median, dtype: float64, 'TRAIN_LIST_MEAN': 0     95.000000
   1     90.766667
   2     90.083333
   3     90.333333
   4     86.016667
   5     87.883333
   6     81.766667
   7     80.916667
   8     77.233333
   9     79.033333
   10    78.250000
   11    84.766667
   Name: train_list_mean, dtype: float64, 'TRAIN_LIST_MIN': 0     92.0
   1     87.0
   2     85.0
   3     83.0
   4     80.0
   5     80.0
   6     77.0
   7     74.0
   8     73.0
   9     74.0
   10    74.0
   11    77.0
   Name: train_list_min, dtype: float64, 'TRAIN_LIST_MAX': 0     98.0
   1     96.0
   2     94.0
   3     92.0
   4     94.0
   5     91.0
   6     89.0
   7     91.0
   8     82.0
   9     87.0
   10    85.0
   11    92.0
   Name: train_list_max, dtyp

### Second Adaption
Expanding TRAIN  - > Train Set always includes all available past values

In [13]:
dict_of_chunk_series_with_expanding_test_and_steady_train = {}

for i, chunk in enumerate(dict_of_chunk_series):
    # acces dataframe of current chunk
    chunk_series_for_chunk = dict_of_chunk_series[chunk]

    # access vital_parameter_series_median of current chunk
    chunk_value_series_for_chunk_median = chunk_series_for_chunk["vital_parameter_series_median"]
    # access vital_parameter_series_mean of current chunk
    chunk_value_series_for_chunk_mean = chunk_series_for_chunk["vital_parameter_series_mean"]
    # access vital_parameter_series_min of current chunk
    chunk_value_series_for_chunk_min = chunk_series_for_chunk["vital_parameter_series_min"]
    # access vital_parameter_series_max of current chunk
    chunk_value_series_for_chunk_max = chunk_series_for_chunk["vital_parameter_series_max"]
        
    # access threshold_high_series of current chunk
    chunk_threshold_high_series_for_chunk = chunk_series_for_chunk["threshold_high_series"]
    # access threshold_low_series of current chunk
    chunk_threshold_low_series_for_chunk = chunk_series_for_chunk["threshold_low_series"]

    # create an empty dictionary for the key of the current chunk
    dict_of_chunk_series_with_expanding_test_and_steady_train[chunk] = {}

    # create multiple test and train lists for that chunk
    for start in range(0, len(chunk_value_series_for_chunk_median) - (TRAIN + TEST)+1, STEP):
        
        # vital_parameter_series_median
        train_list_median = pd.Series(chunk_value_series_for_chunk_median[0: start+TRAIN], name="train_list_median")
        test_list_median = pd.Series(chunk_value_series_for_chunk_median[start+TRAIN : start+TRAIN+TEST], name="test_list_median")
        # vital_parameter_series_mean
        train_list_mean = pd.Series(chunk_value_series_for_chunk_mean[0: start+TRAIN], name="train_list_mean")
        test_list_mean = pd.Series(chunk_value_series_for_chunk_mean[start+TRAIN : start+TRAIN+TEST], name="test_list_mean")
        # vital_parameter_series_min
        train_list_min = pd.Series(chunk_value_series_for_chunk_min[0: start+TRAIN], name="train_list_min")
        test_list_min = pd.Series(chunk_value_series_for_chunk_min[start+TRAIN : start+TRAIN+TEST], name="test_list_min")
        # vital_parameter_series_max
        train_list_max = pd.Series(chunk_value_series_for_chunk_max[0: start+TRAIN], name="train_list_max")
        test_list_max = pd.Series(chunk_value_series_for_chunk_max[start+TRAIN : start+TRAIN+TEST], name="test_list_max")
        
        #threshold series high & low
        threshold_high_for_test_list = pd.Series(chunk_threshold_high_series_for_chunk[start+TRAIN : start+TRAIN+TEST],name="threshold_high_for_test_list")
        threshold_low_for_test_list = pd.Series(chunk_threshold_low_series_for_chunk[start+TRAIN : start+TRAIN+TEST],name="threshold_low_for_test_list")
        
        # For each iteration over the current chunk, we will create a dictionary that holds again the test and train list as dictionary
        # We use the last index of the current train list (which currently refers to the difference to first measurement) as second key
        second_key = train_list_median.index.max()
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key] = {}
        # Assign the train and test list to the current chunk iteration      
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TRAIN_LIST_MEDIAN"] = train_list_median
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TRAIN_LIST_MEAN"] = train_list_mean
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TRAIN_LIST_MIN"] = train_list_min
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TRAIN_LIST_MAX"] = train_list_max

        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TEST_LIST_MEDIAN"] = test_list_median
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TEST_LIST_MEAN"] = test_list_median
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TEST_LIST_MIN"] = test_list_min
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["TEST_LIST_MAX"] = test_list_max

        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["THRESHOLD_HIGH_FOR_TEST_LIST"] = threshold_high_for_test_list
        dict_of_chunk_series_with_expanding_test_and_steady_train[chunk][second_key]["THRESHOLD_LOW_FOR_TEST_LIST"] = threshold_low_for_test_list

In [ ]:
import pickle
# Rausschreiben
third_output_file = open('dict_of_chunk_series_with_expanding_test_and_steady_train.pickle', 'wb')
pickle.dump(dict_of_chunk_series_with_expanding_test_and_steady_train, third_output_file)
third_output_file.close()

In [14]:
dict_of_chunk_series_with_expanding_test_and_steady_train[chunk]

{11: {'TRAIN_LIST_MEDIAN': 0     95.0
  1     90.5
  2     91.0
  3     91.0
  4     85.0
  5     89.0
  6     82.0
  7     80.0
  8     77.0
  9     78.5
  10    78.0
  11    84.0
  Name: train_list_median, dtype: float64, 'TRAIN_LIST_MEAN': 0     95.000000
  1     90.766667
  2     90.083333
  3     90.333333
  4     86.016667
  5     87.883333
  6     81.766667
  7     80.916667
  8     77.233333
  9     79.033333
  10    78.250000
  11    84.766667
  Name: train_list_mean, dtype: float64, 'TRAIN_LIST_MIN': 0     92.0
  1     87.0
  2     85.0
  3     83.0
  4     80.0
  5     80.0
  6     77.0
  7     74.0
  8     73.0
  9     74.0
  10    74.0
  11    77.0
  Name: train_list_min, dtype: float64, 'TRAIN_LIST_MAX': 0     98.0
  1     96.0
  2     94.0
  3     92.0
  4     94.0
  5     91.0
  6     89.0
  7     91.0
  8     82.0
  9     87.0
  10    85.0
  11    92.0
  Name: train_list_max, dtype: float64, 'TEST_LIST_MEDIAN': 12    75.0
  Name: test_list_median, dtype: float64, 'TEST